# Azure AI Document Intelligence 基礎
Azure AI Document Intelligence は、機械学習モデルを使用してドキュメントからキーと値のペア、テキスト、テーブルを抽出するクラウドベースの Azure AI サービスです。本ノートブックでは、[レイアウトモデル](https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-layout) `prebuilt-layout` を使用して PDF からデータを抽出します。最新の API バージョンでは、マークダウンでの出力もサポートされるようになりました。

https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-layout


# 事前準備
この Python サンプルを実行するには、以下が必要です：
- [Azure AI Document Intelligence リソース](https://learn.microsoft.com/azure/ai-services/document-intelligence/create-document-intelligence-resource)。エンドポイントとキーが必要です。
- [Python](https://www.python.org/downloads/release/python-31011/) (この手順はバージョン 3.10.x でテストされています)

これらのデモには、[Visual Studio Code](https://code.visualstudio.com/download) と [Jupyter extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) を使用できます。

## パッケージのインストール

In [ ]:

!pip install azure-identity==1.15.0
!pip install azure-ai-formrecognizer==3.3.2
!pip install jsonpickle

In [ ]:
import azure.ai.formrecognizer
print("azure.ai.formrecognizer", azure.ai.formrecognizer.__VERSION__)

## 必要なライブラリと環境変数のインポート

In [ ]:
from azure.core.credentials import AzureKeyCredential

## 接続設定

In [ ]:
# Azure AI Document Intelligence
document_intelligence_key: str = "<Your document intelligence key>"
document_intelligence_endpoint: str = "<Your document intelligence endpoint>"
document_intelligence_creds: str = AzureKeyCredential(document_intelligence_key)

# PDF から構造を抽出
PDF の OCR に用いる[レイアウトモデル](https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-layout) `prebuilt-layout` は、高度な機械学習ベースのドキュメント分析 API です。これを使用すると、さまざまな形式のドキュメントを受け取り、ドキュメントの構造化されたデータ表現を返すことができます。これは、Microsoft の強力な光学式文字認識 (OCR) 機能の強化バージョンと、ディープラーニング モデルを組み合わせ、テキスト、テーブル、選択マーク、ドキュメント構造を抽出します。

In [ ]:
from azure.ai.formrecognizer import DocumentAnalysisClient
import jsonpickle

# formatting function
def format_polygon(polygon):
    if not polygon:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in polygon])

document_analysis_client = DocumentAnalysisClient(
    endpoint=document_intelligence_endpoint, credential=document_intelligence_creds
)

# URL からの解析の場合
# poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-layout", formUrl)

# sample document
filename = "../../data/源実朝 - Wikipedia.pdf"

with open(filename, "rb") as f:
    poller = document_analysis_client.begin_analyze_document("prebuilt-layout", document = f)

result = poller.result()

### Debug用コード

[jsonpickle](https://pypi.org/project/jsonpickle/) ライブラリを使用すると `AnalyzeResult` オブジェクト構造をそのまま JSON へ保存できます。Document Intelligence の分析データを一旦自分の手元に置いて細かく調べたい時に有用です。

In [ ]:
# Debug 用(AnalyzeResultオブジェクト構造をそのままJSONへ)
json_data = jsonpickle.encode(result)
with open('analyzed_data.json', "w", encoding='utf-8') as f:
    f.write(json_data)

# JSON からオブジェクト構造を復元
# f = open("analyzed_data.json")
# json_str = f.read()
# result = jsonpickle.decode(json_str)

## テキスト行の手書きスタイル
応答では、各テキスト行が手書きスタイルであるかどうかと、信頼度スコアが分類されます。


In [ ]:
for idx, style in enumerate(result.styles):
    print(
        "Document contains {} content".format(
            "handwritten" if style.is_handwritten else "no handwritten"
        )
    )

## ページ
ページ コレクションは、サービス応答に表示される最初のオブジェクトです。レイアウトモデルでは、印刷および手書きのスタイルテキストが `lines` および `words` として抽出されます。このモデルでは、抽出された単語の境界 `polygon` 座標と `confidence` を出力します。

### 選択マーク
ドキュメントから選択マークも抽出されます。抽出された選択マークは、各ページの `pages` コレクション内に示されます。これには、境界 `polygon`、`confidence`、および選択 `state` (`selected/unselected`) が含まれます。関連するテキスト (抽出された場合) も、開始インデックス (`offset`) と `length` として含まれます。`length` はドキュメントのテキスト全体を含む最上位の `content` プロパティを参照します。

In [ ]:
for page in result.pages:
    print("----Analyzing layout from page #{}----".format(page.page_number))
    print(
        "Page has width: {} and height: {}, measured with unit: {}".format(
            page.width, page.height, page.unit
        )
    )

    for line_idx, line in enumerate(page.lines):
        words = line.get_words()
        print(
            "...Line # {} has word count {} and text '{}' within bounding polygon '{}'".format(
                line_idx,
                len(words),
                line.content,
                format_polygon(line.polygon),
            )
        )

        for word in words:
            print(
                "......Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            )

    for selection_mark in page.selection_marks:
        print(
            "...Selection mark is '{}' within bounding polygon '{}' and has a confidence of {}".format(
                selection_mark.state,
                format_polygon(selection_mark.polygon),
                selection_mark.confidence,
            )
        )

## テーブル
レイアウトモデルでは、JSON 出力の `pageResults` セクションにテーブルが抽出されます。抽出されるテーブル情報には、列と行の数、行の範囲、列の範囲が含まれます。境界ポリゴンのある各セルは、その領域が `columnHeader` として認識されているかどうかにかかわらず、情報と共に出力されます。このモデルでは、回転されるテーブルの抽出がサポートされています。各テーブル セルには、行と列のインデックスと境界ポリゴン座標が含まれています。セル テキストの場合、このモデルは開始インデックス (`offset`) を含む `span` 情報を出力します。

In [ ]:
for table_idx, table in enumerate(result.tables):
    print(
        "Table # {} has {} rows and {} columns".format(
            table_idx, table.row_count, table.column_count
        )
    )
    for region in table.bounding_regions:
        print(
            "Table # {} location on page: {} is {}".format(
                table_idx,
                region.page_number,
                format_polygon(region.polygon),
            )
        )
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content,
            )
        )
        for region in cell.bounding_regions:
            print(
                "...content on page {} is within bounding polygon '{}'".format(
                    region.page_number,
                    format_polygon(region.polygon),
                )
            )

## 段落
レイアウト モデルは、`analyzeResults` の最上位オブジェクトとして、`paragraphs` コレクション内の識別されたテキスト ブロックすべてを抽出します。このコレクション内の各エントリはテキスト ブロックを表し、抽出されたテキスト (`content`) と境界 `polygon` 座標を含みます。`span` 情報は、ドキュメントのテキスト全体を含む最上位 `content` プロパティ内のテキストフラグメントを指します。

In [ ]:
for paragraph_idx, paragraph in enumerate(result.paragraphs):
    print(
        "`Paragraph #{}: {}".format(
            paragraph_idx, paragraph.content
        )
    )